##**<AlphaZero를 활용한 틱택토 구현>**
####1. 게임 환경(TicTacToe 클래스)을 강화 학습 알고리즘에 통합
####2. MCTS(몬테카를로 트리 서치)
####3. 신경망(듀엘 네트워크)
####4. 자가 대국(Self-Play)을 포함하는 구조를 설계
####5. 학습 루프(train 코드)

#### **1. 게임 환경(TicTacToe 클래스)을 강화 학습 알고리즘에 통합**

In [276]:
import random
import numpy as np

class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9 # 9개 요소로 이루어진 리스트
        self.current_winner = None  # 승자가 있는 경우 승자('X' 또는 'O')를 저장

    def available_moves(self): # 사용 가능한 위치(보드의 빈 공간)의 목록을 반환
        return [i for i, spot in enumerate(self.board) if spot == ' ']
        # 사용 가능한 인덱스 목록 반환

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ') # 보드의 빈 공간 개수 반환

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter # 'X' 또는 'O'를 저장
            if self.winner(square, letter):
                self.current_winner = letter
            return True # 플레이어가 이번 move로 인해 이겼다면 True 출력
        return False

    def winner(self, square, letter): # 플레이어가 이번 move로 인해 이겼는지 확인
        # 이기는 조건 (행, 열, 또는 대각선으로 3개)
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # 행
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # 열
            [0, 4, 8], [2, 4, 6]              # 대각선
        ]
        for condition in win_conditions:
            if all(self.board[i] == letter for i in condition): # 플레이어가 승리 조건 만족하는지 확인
                return True # 이겼다면 True 출력
        return False

    # AlphaZero에서 상태가 숫자 배열로 처리됨!
    # self.board를 numpy 배열로 변환하는 함수 추가
    def get_state(self):
      state = np.array([1 if x == 'X' else -1 if x == 'O' else 0 for x in self.board])
      return state.reshape(3, 3)  # 3x3 배열 반환

    # 게임 종료 시 1, -1, 또는 0의 보상을 반환
    def reward(self, letter):
      if self.current_winner == letter:
        return 1  # 승리 보상
      elif self.current_winner is None and not self.empty_squares():
        return 0  # 무승부
      else:
        return -1  # 패배 보상

    def print_board(self): # 보드 시각화
        for row in [self.board[i * 3:(i + 1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')


####**2. MCTS(몬테카를로 트리 서치)**
#####(1) 노드 정의: 상태(state), 방문 횟수(N), 누적 보상(W), 확률(Prior Probability)을 포함


In [277]:
class Node:
    def __init__(self, state, parent=None, prior=0):
        self.state = state  # 현재 상태
        self.parent = parent  # 부모 노드
        self.children = {}  # 자식 노드 (행동: 노드)
        self.visits = 0  # 방문 횟수
        self.value_sum = 0  # 누적 가치
        self.prior = prior  # 정책 확률/ 신경망이 예측한 행동의 확률 값 (신경망 출력)

    def get_value(self, epsilon=1.0):
        ucb = epsilon * self.prior * np.sqrt(self.parent.visits) / (1 + self.visits) # 값을 계산하여 자식 노드 선택 시 사용
        return self.value_sum / (1 + self.visits) + ucb

#####(2) 탐색/ 확장/ 시뮬레이션/ 역전파

In [278]:
# mcts_search 함수: 현재 상태에서 MCTS를 실행하여 행동 확률 분포를 반환
def mcts_search(game, model, simulations):
    """
    MCTS를 실행하여 행동 확률 분포를 생성합니다.
    :param game: 현재 TicTacToe 게임 인스턴스
    :param model: 신경망 모델
    :param simulations: MCTS 시뮬레이션 횟수
    :return: 행동 확률 분포 (크기: 9)
    """
    root = Node(state=game.get_state())  # 현재 상태를 루트 노드로 설정

    for _ in range(simulations):
        node = root
        temp_game = TicTacToe()  # MCTS 탐색을 위해 게임을 복사
        temp_game.board = game.board.copy()

        # 1. 탐색 (Select)
        while node.children:
            action, node = max(node.children.items(), key=lambda item: item[1].get_value())

            temp_game.make_move(action, 'X')  # 선택한 행동을 임시 게임에 반영

        # 2. 확장 (Expand)
        if temp_game.empty_squares():
            legal_moves = temp_game.available_moves()
            policy, value = model.predict(temp_game.get_state().reshape(1, 3, 3))

            for move in legal_moves:
                if move not in node.children:
                    next_state = simulate_move(node.state, move)
                    node.children[move] = Node(state=next_state, parent=node, prior=policy[0][move])

            # 확장된 노드 중 하나를 선택
            action, node = max(node.children.items(), key=lambda item: item[1].prior)

            temp_game.make_move(action, 'X')

        # 3. 시뮬레이션 (Simulate)
        reward = temp_game.reward('X') if temp_game.current_winner else value[0][0]

        # 4. 역전파 (Backpropagate)
        backpropagate(node, reward)

    # 행동 확률 계산
    actions = np.zeros(9)  # 행동 확률 분포 (틱택토에서는 9개의 행동)
    for move, child in root.children.items():
        actions[move] = child.visits
    actions /= sum(actions)  # 확률로 정규화

    return actions # 행동 확률 분포 반환 (크기: 9)

In [279]:
# 보조 함수: MCTS에서 특정 행동을 수행한 후의 상태를 반환
def simulate_move(state, move):
    new_state = state.copy()
    row, col = divmod(move, 3)
    new_state[row, col] = 1  # 플레이어의 행동
    return new_state

####**3. 신경망(듀엘 네트워크)**
##### AlphaZero는 정책(Policy)과 값(Value)을 동시에 출력하는 신경망을 사용!
#####입력: 3x3 보드 상태
#####출력: 정책(Policy): 각 행동의 확률 (크기: 9) & 값(Value): 현재 상태의 예상 승률 (크기: 1)


In [280]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TicTcToeModel(nn.Module):
    def __init__(self):
        super(TicTcToeModel, self).__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.policy_head = nn.Linear(64, 9) # 정책 출력 계층
        self.value_head = nn.Linear(64, 1) # 가치 출력 계층

    def forward(self, x):
        x = torch.flatten(x, start_dim=1) # (batch_size, 3, 3) -> (batch_size, 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        policy_output = F.softmax(self.policy_head(x), dim=1)  # 정책 출력 with softmax
        value_output = torch.tanh(self.value_head(x))          # 가치 출력 with tanh
        return policy_output, value_output

    def predict(self, x):
        """배치 입력에 대해 정책 및 가치 예측."""
        self.eval()  # 모델을 평가 모드로 전환
        with torch.no_grad():
            policy, value = self.forward(x)
        return policy, value

# 모델
model = TicTcToeModel()

# 옵티마이저 및 손실 정의
optimizer = torch.optim.Adam(model.parameters())
criterion_policy = nn.CrossEntropyLoss()
criterion_value = nn.MSELoss()

# 손실 계산 예시
dummy_input = torch.rand(4, 3, 3)  # 배치 사이즈 4, 3x3 인
policy_target = torch.randint(0, 9, (4,))  # 정책 더미
value_target = torch.rand(4, 1)  # 가치 더미

# Forward pass
policy_output, value_output = model(dummy_input)

# 손실 계산
policy_loss = criterion_policy(policy_output, policy_target)
value_loss = criterion_value(value_output, value_target)
total_loss = policy_loss + value_loss

# 역전파
optimizer.zero_grad()
total_loss.backward()
optimizer.step()

####**4. 자가 대국(Self-Play)을 포함하는 구조를 설계**
#####AlphaZero는 스스로 대국을 반복하며 데이터를 생성하고, 이를 통해 신경망을 학습함!

In [281]:
def self_play(game, model, mcts_simulations=100):
    data = []

    while game.empty_squares():
        # MCTS 실행
        action_probs = mcts_search(game, model, mcts_simulations)
        action = np.random.choice(len(action_probs), p=action_probs)

        # 데이터 저장
        state = game.get_state()
        data.append((state, action_probs, None))  # 보상은 게임 종료 후 추가

        # 행동 수행
        game.make_move(action, 'X')
        if game.current_winner or not game.empty_squares():
            break

    # 게임 종료 후 보상 업데이트
    reward = game.reward('X')
    for i in range(len(data)):
        data[i] = (data[i][0], data[i][1], reward)
        reward = -reward  # 상대방 보상 반전
    return data

#### **5. 학습 루프(train 코드)**
#####반복적으로 Self-Play를 통해 데이터를 생성하고, 신경망을 업데이트~

In [282]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def train(model, game_class, self_play_fn, optimizer, criterion_policy, criterion_value,
          iterations=10, self_play_games=50, batch_size=32, mcts_simulations=25):
    """
    알파제로 기반 틱택토 학습 루프.

    Args:
        model (nn.Module): TicTcToeModel 모델.
        game_class (class): 틱택토 게임 클래스.
        self_play_fn (func): Self-play 함수.
        optimizer (torch.optim.Optimizer): PyTorch 옵티마이저.
        criterion_policy (nn.Module): 정책 손실 함수.
        criterion_value (nn.Module): 가치 손실 함수.
        iterations (int): 학습 반복 횟수.
        self_play_games (int): Self-play로 생성할 게임 수.
        batch_size (int): 학습 배치 크기.
        mcts_simulations (int): MCTS 시뮬레이션 횟수.
    """
    for iteration in range(1, iterations + 1):
        print(f"=== Iteration {iteration}/{iterations} ===")

        # Step 1: Self-play로 데이터 생성
        all_data = []
        for _ in range(self_play_games):
            game = game_class()
            data = self_play_fn(game, model, mcts_simulations)
            all_data.extend(data)

        # Step 2: 데이터 준비
        states, policies, rewards = zip(*all_data)
        states = torch.from_numpy(np.array(states)).float()  # (batch_size, 3, 3)
        policies = torch.tensor(policies, dtype=torch.float32)  # (batch_size, 9)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)  # (batch_size, 1)

        # Step 3: DataLoader로 배치 처리
        dataset = TensorDataset(states, policies, rewards)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Step 4: 모델 학습
        model.train()
        total_policy_loss, total_value_loss = 0, 0

        for batch_states, batch_policies, batch_rewards in dataloader:
            optimizer.zero_grad()

            # Forward pass
            policy_output, value_output = model(batch_states)

            # 손실 계산
            policy_loss = criterion_policy(policy_output, batch_policies)
            value_loss = criterion_value(value_output, batch_rewards)
            loss = policy_loss + value_loss

            # 역전파 및 옵티마이저 스텝
            loss.backward()
            optimizer.step()

            # 손실 기록
            total_policy_loss += policy_loss.item()
            total_value_loss += value_loss.item()

        # 평균 손실 출력
        avg_policy_loss = total_policy_loss / len(dataloader)
        avg_value_loss = total_value_loss / len(dataloader)
        print(f"Policy Loss: {avg_policy_loss:.4f}, Value Loss: {avg_value_loss:.4f}")


In [283]:
# 모델, 옵티마이저, 손실 정의
model = TicTcToeModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion_policy = nn.MSELoss()  # 정책 출력에 맞게 설정
criterion_value = nn.MSELoss()

# 학습 실행
train(
    model=model,
    game_class=TicTacToe,  # TicTacToe 게임 클래스 전달
    self_play_fn=self_play,  # Self-play 함수 전달
    optimizer=optimizer,
    criterion_policy=criterion_policy,
    criterion_value=criterion_value,
    iterations=10,  # 학습 반복 횟수
    self_play_games=50,  # Self-play로 생성할 게임 수
    batch_size=32,  # 학습 배치 크기
    mcts_simulations=25  # MCTS 시뮬레이션 횟수
)


=== Iteration 1/10 ===


TypeError: flatten(): argument 'input' (position 1) must be Tensor, not numpy.ndarray